In [1]:
from openai import OpenAI

client = OpenAI()
text = {
        "format": {
            "type": "json_schema",
            "name": "repsponse_json_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "statements": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": { "type": "string" },
                                "output": { "type": "string" }
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    }
                },
                "required": ["statements"],
                "additionalProperties": False
            },
            "strict": True
        }
    }

Function to Schema


In [2]:

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Retrieves current weather for the given location.",
    "strict": True,
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            },
            "units": {
                "type": ["string", "null"],
                "enum": ["celsius", "fahrenheit"],
                "description": "Units the temperature will be returned in."
            }
        },
        "required": ["location", "units"],
        "additionalProperties": False
    }
},
{
    "type": "function",
    "name": "get_time",
    "description": "Retrieves current time for the given location.",
    "strict": True,
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            },
            "timezone": {
                "type": ["string", "null"],
                "description": "Timezone to convert the time to."
            }
        },
    "required": ["location", "timezone"],
    "additionalProperties": False
}
}
]

In [3]:
from openai import OpenAI
import logging
client = OpenAI()
logging.basicConfig(
    level=logging.INFO,  # or logging.DEBUG for more detail
    format='%(asctime)s - %(levelname)s - %(message)s'
)

Initialize Query Tool 


In [4]:

import pandas as pd
from tools.query_dataframe import QueryDataFrameTool

df = pd.read_csv(r"C:\Users\AakashAI\Desktop\Repositories\Sales Agent\cleaned_leads.csv")
QUERY_DATAFRAME_TOOL_INSTRUCTIONS = open(r"prompts\query_dataframe_tool\prompt_v1.txt", encoding="utf-8").read()
query_tool = QueryDataFrameTool(df, QUERY_DATAFRAME_TOOL_INSTRUCTIONS)
# query_tool.query_dataframe("how many leads are there in the dataset?")

Tool Definition

In [ ]:
from utils.function_to_schemas import function_to_schema
from tools.query_dataframe import QueryDataFrameInput, QueryDataFrameOutput
# query dataframe method is used for tool invocations
tools = {"query_dataframe": query_tool.query_dataframe}
tool_models = {"query_dataframe": QueryDataFrameInput}
tool_schemas = [function_to_schema(query_tool.query_dataframe, QueryDataFrameInput)]

<bound method QueryDataFrameTool.query_dataframe of <tools.query_dataframe.QueryDataFrameTool object at 0x000002721495E720>>


In [4]:
tool_schemas

[{'type': 'function',
  'name': 'query_dataframe',
  'description': 'Main method to query the DataFrame.',
  'strict': True,
  'parameters': {'properties': {'user_query': {'description': 'The natural language query to be executed on the DataFrame.',
     'title': 'User Query',
     'type': 'string'}},
   'required': ['user_query'],
   'title': 'QueryDataFrameInput',
   'type': 'object',
   'additionalProperties': False}}]

In [ ]:
from agent.agent import Agent
SDR_ASSISTANT_INSTRUCTIONS = open(r"prompts\sdr_agent\prompt_v1.txt", encoding="utf-8").read()
agent = Agent("SalesAgent", client, SDR_ASSISTANT_INSTRUCTIONS, tool_schemas, tool_models, tools)

In [8]:
agent.response("can you show leads which are not converted from these leads?")


2025-04-23 22:46:10,550 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Function call detected


2025-04-23 22:46:12,468 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-04-23 22:46:12,484 - INFO - Question: user_query='Show leads from Bombay with source Landing Page that are not converted. Include their names and relevant details.'


LLM response: Response(id='resp_6809205ab06481918036820a2e85f1d40d02ea061b34d0ac', created_at=1745428570.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseOutputMessage(id='msg_6809205afbe081918ae8b3b582f9c5ad0d02ea061b34d0ac', content=[ResponseOutputText(annotations=[], text='{"explanation":"Query rephrased for available city value.","output":"df[(df[\'City\'] == \'Mumbai\') & (df[\'Lead Source\'] == \'Landing Page\') & (df[\'Converted\'] == 0)]"}', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatTextJSONSchemaConfig(name='response_json_schema', schema_={'type': 'object', 'properties

2025-04-23 22:46:13,134 - INFO - Running PandasAI with openai LLM...
2025-04-23 22:46:13,146 - INFO - Prompt ID: 2d40050d-4e62-45e6-bdbe-c9330a3010af


Query result: status='success' explanation="PandasAI returned a scalar: Unfortunately, I was not able to get your answers, because of the following error:\n\nexpected string or bytes-like object, got 'QueryDataFrameInput'\n" data="Unfortunately, I was not able to get your answers, because of the following error:\n\nexpected string or bytes-like object, got 'QueryDataFrameInput'\n" error=None
Is empty: False


2025-04-23 22:46:30,047 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


"It seems there was a technical issue while trying to retrieve the data. Let me try again.\n\nI will fetch the list of leads from Bombay with source Landing Page who have not converted.Let me retry fetching that information for you.I'm working on gathering this information. Let me give it another try for you.Let me attempt to retrieve the details once more for you.Just a moment, I'll retrieve those details for you again.Let's try to fetch this data one more time. Please hold on.I'll attempt to retrieve the information again. Hang on just a moment.I'm on it. I'll try to gather that data once more.Let's try that again. I'll attempt to retrieve the data.Let me give it another shot to retrieve the information. Hang tight!Allow me to try retrieving that information again for you.I'll give it another attempt to get that data for you. Please hold on.I'll try fetching the data once more. Please bear with me.I'll make another attempt to retrieve the information. Just a moment!Let me try fetchin

In [10]:
agent.memory

[{'role': 'developer',
  'content': 'You are a friendly and efficient assistant for a Sales Development Representative (SDR).\nALWAYS CHECK FOR INFORMATION IN THE CONVERSATION HISTORY BEFORE ASKING THE SDR FOR MORE INFORMATION. TOOLS MIGHT HAVE BEEN CALLED ALREADY.\n\nYour job is to support the SDR in identifying high-quality leads and drafting effective, personalized sales emails.\nYou help the SDR save time by interpreting their natural language requests and converting them into meaningful actions using the tools available to you.\n\nYou have access to three tools:\n\n1. query_dataframe\n   - Use this tool to retrieve relevant leads from the database.\n   - NEVER write pandas code yourself. Instead, rephrase the SDR’s request in natural language and call this tool.\n   - Use this tool when the SDR asks to find or filter leads (e.g., "Show me leads from Mumbai who haven’t converted yet").\n   - Give names of the leads when the SDR asks for a list of leads along with their details.\n\n

In [8]:
def load_prompt(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()
prompt_template = load_prompt(r".\prompts\query_dataframe_tool\prompt_v1.txt")
prompt_template

'You are an expert data analyst.\n\n    You have access to a tool called `query_dataframe` that accepts a Python expression as input and runs it on a DataFrame called `df`.\n\n    Here is the DataFrame schema:\n    {schema_description}\n\n    Below are a few one-shot examples of valid filtering queries:\n\n    Example 1:\n    User Request: "List all leads from City \'Metropolis\' that have not been converted"\n    Expected tool call: query_dataframe("df[(df[\'City\'] == \'Metropolis\') & (df[\'Converted\'] == 0)]")\n\n    Example 2:\n    User Request: "Show me leads from Source \'Web\' with a lead score above 100"\n    Expected tool call: query_dataframe("df[(df[\'Lead Source\'] == \'Web\') & (df[\'Lead Score\'] > 100)]")\n\n    Example 3:\n    User Request: "Find leads that are qualified and in the age group \'30-35\'"\n    Expected tool call: query_dataframe("df[(df[\'Lead Stage\'] == \'Qualified\') & (df[\'Age\'] == \'30-35\')]")\n\n\n    Now, a user has asked to filter the DataFram

In [7]:
schema_description ="HELLO WORLD"
user_request = "What is the weather in Paris today?"
prompt = prompt_template.format(
    schema_description=schema_description,
    user_request=user_request
)


You are an expert data analyst.

    You have access to a tool called `query_dataframe` that accepts a Python expression as input and runs it on a DataFrame called `df`.

    Here is the DataFrame schema:
    HELLO WORLD

    Below are a few one-shot examples of valid filtering queries:

    Example 1:
    User Request: "List all leads from City 'Metropolis' that have not been converted"
    Expected tool call: query_dataframe("df[(df['City'] == 'Metropolis') & (df['Converted'] == 0)]")

    Example 2:
    User Request: "Show me leads from Source 'Web' with a lead score above 100"
    Expected tool call: query_dataframe("df[(df['Lead Source'] == 'Web') & (df['Lead Score'] > 100)]")

    Example 3:
    User Request: "Find leads that are qualified and in the age group '30-35'"
    Expected tool call: query_dataframe("df[(df['Lead Stage'] == 'Qualified') & (df['Age'] == '30-35')]")


    Now, a user has asked to filter the DataFrame with this request:
    'What is the weather in Paris tod

In [4]:
agent.memory

[{'role': 'developer', 'content': 'You are a helpful agent'},
 {'role': 'user', 'content': "What's the weather in Paris today?"},
 {'type': 'function_call_output',
  'call_id': 'call_ACwMwh8MU7cJWcpgD16Bivli',
  'output': "Error: 'str' object has no attribute 'location'"}]

In [ ]:
agent.tools

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Retrieves current weather for the given location.',
  'strict': True,
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'City and country e.g. Bogotá, Colombia'},
    'units': {'type': ['string', 'null'],
     'enum': ['celsius', 'fahrenheit'],
     'description': 'Units the temperature will be returned in.'}},
   'required': ['location', 'units'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'get_time',
  'description': 'Retrieves current time for the given location.',
  'strict': True,
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'City and country e.g. Bogotá, Colombia'},
    'timezone': {'type': ['string', 'null'],
     'description': 'Timezone to convert the time to.'}},
   'required': ['location', 'timezone'],
   'additionalProperties': False}}]